In [1]:
import googlemaps
import pandas as pd
import numpy as np
from urllib.parse import quote

In [2]:
def get_next_destination(origin:str, distance_matrix:pd.DataFrame):
    """
    This function returns the best consequent given an starting point. It selects the destination with the minimum distance/time
    and then remove that location so the value is not duplicated.

    :param origin: is the starting point from where we are going to look for the next destination.

    :param distance_matrix:  is a squared matrix with the distance/time between different locations. Column names have the Origin,
    while rows have the destinations. This parameter is then returned without the destination selected.
    """

    id_min = distance_matrix[distance_matrix[origin]>0][origin].idxmin()
    destination = distance_matrix.loc[id_min, 'destination']
    distance_matrix = distance_matrix[distance_matrix['destination'] != destination]
    
    return destination, distance_matrix

In [3]:
def get_best_route(start_point:str, distance_matrix:pd.DataFrame):
    """
    This function returns the best route based on a starting point, which is on the distance matrix provided.
    In order to return a route it uses a heuristic method.

    :param start_point: is the starting point of the travel, where the travel begin

    :param distance_matrix: is a squared matrix with the distance/time between different locations. Column names have the Origin,
    while rows have the destinations.
    """

    best_consequent = [start_point]
    locations = distance_matrix.iloc[:,1:].columns.to_list()
    locations = [x for x in locations if x != start_point]

    for i in range(1, len(locations)+1):
        origin = best_consequent[i - 1]
        destination, distance_matrix = get_next_destination(origin, distance_matrix)
        best_consequent.append(destination)
    
    return best_consequent

In [4]:
def get_distance_matrix(locations:list, gmaps):
    """
    
    """
    result = gmaps.distance_matrix(mode='driving', origins=locations, destinations=locations, region='AR', units='metric')

    raw_data = []
    for i, origin in enumerate(locations):
        for j, destination in enumerate(locations):
            data = {
                'origin': origin,
                'destination': destination,
                'distance': result['rows'][i]['elements'][j]['distance']['value'],
                'duration': result['rows'][i]['elements'][j]['duration']['value'],
                'status':result['rows'][i]['elements'][j]['status']
            }

            # Agregar el diccionario a la lista
            raw_data.append(data)

    df = pd.json_normalize(raw_data)

    # ACA DEBERÍAMOS CHEQUEAR QUE TODOS LOS VALORES DEL DF TENGAN EL STATUS = 'OK'

    return df.pivot_table(index='destination', columns='origin', values='duration').reset_index()

In [5]:
def get_url_route(best_route:list, gmaps):
    """
    This function returns the URL to the route generated.

    :param best_route: is a list with all the locations in order to generate the route.

    :param gmaps: is the google maps client. A ESTO DEBERÍAMOS HACERLO TIPO VARIABLE GLOBAL DESPUES
    """
    start = best_route[0]
    end = best_route[len(best_route)-1]
    way_points = best_route[1:len(best_route)-1]

    route = gmaps.directions(origin=start, destination=end, waypoints=way_points, mode="driving", alternatives=False, units="metric", region="AR", optimize_waypoints=False)

    overview_polyline = route[0]['overview_polyline']['points']

    # fix locations to create the route URL
    url_start       = quote(start, safe='')
    url_end         = quote(end, safe='')
    url_waypoints   = "|".join(quote(wp, safe='') for wp in way_points)

    return f"https://www.google.com/maps/dir/?api=1&origin={url_start}&destination={url_end}&waypoints={url_waypoints}&travelmode=driving&dir_action=navigate&waypoints={overview_polyline}"

In [11]:
gmaps = googlemaps.Client(key='')

# el start point debería estar indicado por la empresa, sería el punto de acceso a la ciudad final. 
# Sino también podemos tomar buscar la dirección del acceso a la ciudad final y tomarlo como un punto extra
start_point = 'Pablo Stampa 2510, Chajari'
# esta lista debería venir de una query a la base de datos trayendo todos los destinos para un viaje

locations = ['9 de Julio 3151, Chajari', 'Saenz Peña 3695, Chajari', 'Chaco 2340, Chajari', 'Saenz Peña 2665, Chajari']

# esto puede ir como no, dependiendo de la solución que tomemos.
locations.append(start_point)

distance_matrix = get_distance_matrix(locations, gmaps)

# Remove the start point as a posible destination because it is, indeed, the start point
distance_matrix = distance_matrix[distance_matrix['destination'] != start_point]

best_route = get_best_route(start_point, distance_matrix)

url = get_url_route(best_route, gmaps)

In [16]:
url

'https://www.google.com/maps/dir/?api=1&origin=Pablo%20Stampa%202510%2C%20Chajari&destination=Saenz%20Pe%C3%B1a%203695%2C%20Chajari&waypoints=Saenz%20Pe%C3%B1a%202665%2C%20Chajari|9%20de%20Julio%203151%2C%20Chajari|Chaco%202340%2C%20Chajari&travelmode=driving&dir_action=navigate&waypoints=vtuzDtq{`JvF|G\\b@xCpDxBmC|F{GpCoDh@m@Pc@Qb@i@l@bHnInCbDnF~GyBfCwDnEsEtFeCvCcEaF{@cAe@h@|BnCfCxCa@f@_ClCkB`CwDtEgDxDyF`Hi@r@BHFJlHzI`HzIs@~@w@|@]`@GRDP~@nAJH^@^QVWrA_BbGmHvEb@A@?@AB?D@LHDJ?HI@KEMBGxAqBrCiDdBuBtF{GfNmPlDeEzBsC{BqCmFqGII'